In [2]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time

In [ ]:
#hidden token

In [4]:
def auth():
    return os.getenv('TOKEN')
def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

In [37]:
def create_url(start_date, end_date, max_results):
    
    search_url = "https://api.twitter.com/2/users/396045469/tweets" #Endpoint we collect data from

    #change params based on the endpoint you are using
    query_params = {
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'exclude': 'retweets,replies',
                    'tweet.fields': 'created_at',
                    'next_token': {}}
    return (search_url, query_params)

In [38]:
def connect_to_endpoint(url, headers, params, next_token = None):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()

In [39]:
#Inputs for the request
bearer_token = auth()
headers = create_headers(bearer_token)
start_time = "2021-01-01T00:00:00Z"
end_time = '2021-12-01T00:00:00Z'
max_results = 100

In [40]:
url = create_url(start_time, end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

print(json.dumps(json_response, indent=4, sort_keys=True))


Endpoint Response Code: 200
{
    "data": [
        {
            "created_at": "2021-11-30T13:58:33.000Z",
            "id": "1465681659634884612",
            "text": "$MANA https://t.co/SS7Dm3xDJd"
        },
        {
            "created_at": "2021-11-30T12:47:38.000Z",
            "id": "1465663815396302859",
            "text": "$SOL + @Grayscale https://t.co/XT3vKhdPWs"
        },
        {
            "created_at": "2021-11-30T01:08:15.000Z",
            "id": "1465487807556853760",
            "text": "Digital Currency Group moving its headquarters from NYC to CT \n\n@WTNH @KenHoustonTV1 \n\nhttps://t.co/dEkPMmSBpI"
        },
        {
            "created_at": "2021-11-29T18:37:40.000Z",
            "id": "1465389514101305348",
            "text": "Digital Currency Group Relocating Headquarters to Stamford via @NBCConnecticut \nhttps://t.co/ig0qm1AQRE"
        },
        {
            "created_at": "2021-11-29T16:56:42.000Z",
            "id": "1465364106756698119",
       

In [9]:
# Create file
csvFile = open("data.csv", "a", newline="", encoding='utf-8')
csvWriter = csv.writer(csvFile)

csvFile.close()

In [21]:
def append_to_csv(json_response, fileName):

    #A counter variable
    counter = 0

    #Open OR create the target CSV file
    csvFile = open(fileName, "a", newline="", encoding='utf-8')
    csvWriter = csv.writer(csvFile)

    #Loop through each tweet
    for tweet in json_response['data']:
        
        # We will create a variable for each since some of the keys might not exist for some tweets
        # So we will account for that

        # 4. Tweet ID
        tweet_id = tweet['id']
        
        # 2. Time created
        created_at = dateutil.parser.parse(tweet['created_at'])

        # 8. Tweet text
        text = tweet['text']
        
        # Assemble all data in a list
        res = [tweet_id, text, created_at]
        
        # Append the result to the CSV file
        csvWriter.writerow(res)
        counter += 1

    # When done, close the CSV file
    csvFile.close()

    # Print the number of tweets for this iteration
    print("# of Tweets added from this response: ", counter) 

In [41]:
append_to_csv(json_response, 'data.csv')

# of Tweets added from this response:  100
